## Clustering to clean H1B Job Titles

The goal of this project is to try to replicate the clustering functionality of Google's OpenRefine software. The idea is that in some data fields, unstructured entries that are spelled differently, etc., may really mean the same thing. 

First, let's import the necessary packages.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

import sklearn.cluster
import distance

The data is taken from governmental records of applications for HB1 visas. You can find it here: https://nyu.app.box.com/s/9oz3qx886zpwwfm6ewj89pvjuee2eqp5. I saved it with a csv extension in Sublime.

First, let's load the data into a dataframe so we can steal the column we want.

In [2]:
df = pd.read_csv("H1B.csv")
df.dtypes

SUBMITTED_DATE           object
CASE_NO                  object
NAME                     object
ADDRESS                  object
ADDRESS2                 object
CITY                     object
STATE                    object
POSTAL_CODE              object
NBR_IMMIGRANTS            int64
BEGIN_DATE               object
END_DATE                 object
JOB_TITLE                object
DOL_DECISION_DATE        object
CERTIFIED_BEGIN_DATE     object
CERTIFIED_END_DATE       object
JOB_CODE                  int64
APPROVAL_STATUS          object
WAGE_RATE_1             float64
RATE_PER_1               object
MAX_RATE_1              float64
PART_TIME_1              object
CITY_1                   object
STATE_1                  object
PREVAILING_WAGE_1       float64
WAGE_SOURCE_1            object
YR_SOURCE_PUB_1         float64
OTHER_WAGE_SOURCE_1      object
WAGE_RATE_2             float64
RATE_PER_2               object
MAX_RATE_2              float64
PART_TIME_2              object
CITY_2  

We're going to be working with job titles. First, let's take a look at what our job titles look like so we can understand the problem. We'll group the dataframe by titles, and then extract each one to a numpy array. Since the distance formula we're going to use is a bit computationally expensive, we'll only use the first 300 groups (jobs).

In [19]:
titles = df.groupby('JOB_TITLE')

In [20]:
jobs = []
counter = 0
limit = 300

for group in titles.groups:
    if group not in jobs:
        jobs.append(group)
        counter += 1
    if counter >= limit:
        break

jobs_array = np.asarray(jobs)

In [21]:
jobs_array[:20]

array(['Software Engineer (Consultant)',
       'Software Engineer (Software Development Director)',
       'Assistant VP - Economist',
       'VICE PRESIDENT & CHIEF OPERATING OFFICER', 'PHYSICIAN RESIDENT',
       'Network Manager', 'IT Architect', 'BUSINESS DEVELOPMENT MANAGER ',
       'PostDoctoral Fellow', 'Adjunct Trainer',
       'PATENT SPECIALIST(Chemical Arts)', 'Staff Research Associate',
       'SR. FINANCIAL TECHNOLOGY ADVISOR', 'Software Project Engineer',
       'COMPUTER SUPPORT SPECIALIST', 'PGY 4 Medical Resident/Fellow ',
       'DENTAL OFFICE MANAGER AND DENTAL ASSISTANT', 'PROGAMMER ANALYST',
       'Web Applications Developer', 'Computer Systems Administrator'], 
      dtype='|S50')

As you can see if you scroll through the list, even once we've taken the unique titles out of the dataframe, there are tons of overlapping positions. There are lower case and upper case, words switched around, misspellings, etc. If we want to make this data useful and visualize it, we'll need to clean this up.

First, we can change all of the terms to lowercase. We can argue that it's a good idea to keep the punctuation, but we'll remove it to make it easier on the clustering later on.

In [22]:
#Convert all titles to lower case

for i in range(len(jobs_array)):
    jobs_array[i] = jobs_array[i].lower()

#Strip punctuation

for i in range(len(jobs_array)):
    jobs_array[i] = jobs_array[i].strip('/.,:;-– ')

Ok, now our data is ready for clustering.

The way to calculate the similarity between strings is called the "Levenshtein Distance." Code borrowed from http://stats.stackexchange.com/questions/123060/clustering-a-long-list-of-strings-words-into-similarity-groups. More info on the distance formula here: https://rosettacode.org/wiki/Levenshtein_distance#Python.

In [23]:
lev_similarity = -1 * np.array([[distance.levenshtein(j1,j2) for j1 in jobs_array] for j2 in jobs_array])

So we've created a matrix (in array form) of the Levenshtein Distance of each job title from the other job titles in the original jobs array. Here's what it looks like:

In [24]:
lev_similarity

array([[  0, -25, -23, ..., -19, -18, -28],
       [-25,   0, -40, ..., -38, -36, -37],
       [-23, -40,   0, ..., -17, -18, -22],
       ..., 
       [-19, -38, -17, ...,   0, -12, -24],
       [-18, -36, -18, ..., -12,   0, -25],
       [-28, -37, -22, ..., -24, -25,   0]])

Now we'll cluster these values. Affinity Propagation seems to be the right algorithm for the job, since we've already calculated the Levenshtein Distances for our jobs array. The algorithm was first proposed for this purpose here:http://science.sciencemag.org/content/315/5814/972.

Affinity Propagation seems similar to K-Means, but instead of clustering and then re-iterating, the algorithm sends messages from data to other data to figure out what's close and what's not. K-Means, on the other hand, chooses random centroids (not the case in AP) and then figures out which points are closest. Info from here: http://www.psi.toronto.edu/affinitypropagation/faq.html.

Another super important (and helpful) feature of Affinity Propagation is that we don't need to specify the number of centroids / exemplars, which is key for the nature of our data set.

In [25]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping = 0.5)
affprop.fit(lev_similarity)

AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

Now that we fit the model, let's print out all of the clusters into a Pandas series (so we can index by a string).

In [26]:
def orderClusters(array):
    
    clusters = pd.Series()
    
    for cluster_id in np.unique(affprop.labels_):

        exemplar = array[affprop.cluster_centers_indices_[cluster_id]]

        cluster = np.unique(array[np.nonzero(affprop.labels_==cluster_id)])

        if exemplar not in clusters:
            clusters[exemplar] = cluster
            
    return clusters

In [27]:
clusters = orderClusters(jobs_array[:500])

counter = 0
for key in clusters.keys():
    print key , ': ' , clusters[key]
    counter += 1
    if counter >= 10:
        break

software engineer (software development director) :  ['software engineer (software development director)']
vice president & chief operating officer :  ['vice president & chief operating officer'
 "vice president, int'l compensation & benefits"]
physician resident :  ['industrial designer-automotive' 'mathematician and math modeler'
 'pgy 4 medical resident/fellow' 'physical therapy assistant'
 'physician resident' 'physician, internal medicine' 'physician/internist'
 'production designer']
postdoctoral fellow :  ['post doctoral fellow' 'post doctoral fellow in zoology'
 'postdoctoral appointee' 'postdoctoral fellow' 'postdoctoral scholar'
 'postdoctoral scientist' 'postgraduate researcher iv']
computer support specialist :  ['computer specialist' 'computer support specialist'
 'tecnical support specialist' 'title computer support specialist']
dental office manager and dental assistant :  ['dental office manager and dental assistant'
 'hotel culinary manager pastry (assistant)']
progamm

Definitely not bad for a first run through! A lot of these make sense and probably caught some real errors.

If we adjust the "preference" argument, we can force the algorithm to employ more clusters. To see how changing the preference changes our cluster size, we'll run it a few different times to find the length of the dictionary (i.e. the number of exemplars).

In [28]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping = 0.5, preference = -10)
affprop.fit(lev_similarity)
clusters = orderClusters(jobs_array)
len(clusters)

176

In [38]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping = 0.5, preference = -3)
affprop.fit(lev_similarity)
clusters = orderClusters(jobs_array)
len(clusters)

286

The total number of data points that we grabbed is close to being reached, so we can see that as the preference approaches 0, we approach no clusters.

Now let's print out all of the clusters that have more than one member.

In [39]:
clusters = orderClusters(jobs_array)

for key in clusters.keys():
    if len(clusters[key]) >= 2:
        print key , ': ' , clusters[key]

postdoctoral fellow :  ['post doctoral fellow' 'postdoctoral fellow' 'postdoctoral scholar']
manager/biostatistics :  ['manager , biostatistics' 'manager/biostatistics']
programmer/analyst :  ['progammer analyst' 'programmer/analyst']
development system architect :  ['development system architect' 'development systems architect']
application engineer :  ['application engineer' 'application engineer i']
software design engineer/level 62 :  ['software design engineer 1 & 2' 'software design engineer/level 62'
 'software design engineer/level 64']
account executive i :  ['account executive i' 'account exexcutive']
sr software development engineer :  ['software development engineer' 'sr software development engineer']
systems/software engineer :  ['systems software engineer' 'systems/software engineer']


So how the heck are we supposed to know how many clusters are correct = what preference to use? Well, that's a great question, and the subject of this exact research paper from Cornell: https://arxiv.org/abs/0805.1096. The basic idea is – keep iterating until you converge on the right amount of clusters. It's called "Adaptive Affinity Propagation."

Until the scientists figure that out, let's take our clusters and update the dataframe by replacing any values in the cluster with the exemplars.

In [40]:
replacements = []

def replaceClusterMember(x):
    replaced = False
    
    #Run through each cluster / exemplar pair and see if the cell value is in it
    
    for key in clusters.keys():
        if x in clusters[key]:
            #If it is, add it to the new list
            replacements.append(key)
            replaced = True
    #If it isn't, add the existing value
    if replaced == False:
        replacements.append(x)

df["JOB_TITLE"].apply(lambda x: replaceClusterMember(x))

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
9969    None
9970    None
9971    None
9972    None
9973    None
9974    None
9975    None
9976    None
9977    None
9978    None
9979    None
9980    None
9981    None
9982    None
9983    None
9984    None
9985    None
9986    None
9987    None
9988    None
9989    None
9990    None
9991    None
9992    None
9993    None
9994    None
9995    None
9996    None
9997    None
9998    None
Name: JOB_TITLE, dtype: object

In [41]:
df["Cleaned_Title"] = replacements

### Conclusions

1) To improve the accuracy of this project, we'd need to implement and adaptive framework to find the right preference value (= the optimal number of clusters).

2) For full effectiveness, we'd need to run the distance formula and clustering algorithm on every group, which would require some distributed computing on a cluster.